## 音声認識モデル Wav2Vec を SageMaker 上でデプロイして試してみる



本チュートリアルでは、音声認識モデルの１つである wav2vec2 のモデルを Studio Lab 経由で、AWS 環境上にデプロイする流れを体験してみます。  
Amazon SageMaker 上で Hugging Face Inference DLC を使って MetaAI の [wav2vec2](https://arxiv.org/abs/2006.11477) モデルを手軽に利用することが可能です。  

このサンプルでは、
- Studio Lab 上で Amazon SageMaker の機能を利用する設定を行う
- Studio Lab 経由で Amazon SageMaker 上に `transformers` のモデルをデプロイする
- デプロイしたモデルに対して音声ファイルを投げて推論させてみる

の３つをおこなっていきます。  
なお、このサンプルは [Automatic Speech Recogntion with Hugging Face's Transformers & Amazon SageMaker](https://github.com/huggingface/notebooks/blob/main/sagemaker/20_automatic_speech_recognition_inference/sagemaker-notebook.ipynb) をベースに作成しています。

## 環境のセットアップ


下記のハンズオン資料の「2-2. AWS へ接続するための環境構築」「2-3. SageMaker Training Instance が利用する IAM ロールを作成する」で紹介されている手順とほぼ同じです。  

- https://github.com/aws-samples/aws-ml-enablement-workshop/blob/main/notebooks/scenario_churn/customer_churn_sagemaker.ipynb 

### モジュールのインストール

インストールが完了したら、カーネルを再起動することを忘れないでください。  

In [1]:
!pip install -U boto3
!pip install -U sagemaker
!pip install -U transformers

You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python -m pip install --upgrade pip' command.



### IAM ユーザーの作成と認証情報の設定
今回のサンプルでは、Studio Lab で動かしている Notebook 上ではなく AWS 環境上で Whisper をデプロイして使い方を確かめてみます。そのためには、Notebook から AWS 環境にアクセスする必要があるためその認証情報をこれから設定します。

IAM ユーザーを作成し、そこから得られるアクセスキーとシークレットキーを登録します。

まずは、AWS のコンソール画面を開いて左上の検索窓で「IAM」と検索します。トップに出てくる IAM をクリックして IAM のサービスページを開きます。

![](./imgs/001_IAM_search.png)

左側メニューから、「ユーザー」をクリックして IAM ユーザーの設定画面に遷移します。

![](./imgs/002_IAM_user.png)

次に、「ユーザーを追加」をクリックしてユーザーの作成を開始します。

![](./imgs/003_user_create.png)

ユーザー名に「whisper-sample-user」（他の名称でも大丈夫です）、「アクセスキー - プログラムによるアクセス」にチェックをつけます。

![](./imgs/004_user_info.png)

「次のステップ」をクリックします。  
その後「既存のポリシーを直接アタッチ」を選択し、ポリシーの検索で「SageMakerFullAccess」と入力します。そうすると「AWSSageMakerFullAccess」のポリシー候補が現れるのでこれを選択します。

![](./imgs/005_user_policy.png)

次に、検索窓に「PowerUserAccess」と検索し候補に出てきた「PowerUserAccess」を選択します。  

![](./imgs/006_user_poweruseraccess.png)

「次のステップ」をクリックするとタグの設定画面が出てきますが、ここは特に入力せずにスキップします。  

これまでに設定した項目の確認ページが出てくるので問題なければ「ユーザーの作成」をクリックします。

![](./imgs/007_user_confirmation.png)  

無事ユーザーが作成されるとユーザーキーとシークレットキーが表示されるのでメモに残しておきます。これらの情報を使って Studio Lab 経由で AWS 環境にアクセスを行います。  
**ここで取得されるクレデンシャル情報の扱いには十分注意してください**。

![](./imgs/008_user_credentials.png)


次に、 Studio Lab の画面に戻って先ほど取得したアクセスキーなどの情報を登録していきます。  

画面上部のメニューから 「File -> New -> Terminal」 と選択してターミナルの起動をします。  
![](./imgs/009_start_terminal.png)

開かれたターミナルで `aws configure` を実行します。
そこでアクセスキーとシークレットキーを聞かれるので先ほどメモした値を入力します。  

![](./imgs/010_aws_configure.png)


### SageMaker Inference Instance が使用する IAM ロールを作成する

今度は、SageMaker 側でモデルをデプロイする際にデプロイされたインスタンスに付与される IAM ロールを作成します。  


AWS のコンソール画面に戻ります。  
先ほどと同様の手順で IAM のサービス画面を開き、「ロール」を左側のメニューから選択します。 
IAM ロールの画面が開かれたら「ロールの作成」ボタンをクリックします。  

![](./imgs/011_role_create.png)  

ロールの作成画面が表示されたら信頼されるエンティティタプとして「AWS のサービス」を選択し、ユースケースのところは下の検索欄から「SageMaker」などと検索して SageMaker を選択します。

![](./imgs/012_role_entity.png)

「次へ」をクリックし、「AmazonSageMakerFullAccess」のポリシーがアタッチされていることを確認します。  

![](./imgs/013_role_policy.png)

「次へ」をクリックし、Role 名を設定します。「StudioLabWhisperExecutionRole」と入力し、他の項目はいじらずに「ロールを作成」をクリックします。

![](./imgs/014_role_name.png)

作成した IAM ロールのリソースネームである ARN を取得します。  
IAM ロールの画面から、検索欄で「StudioLabWhisper」などと入力して先ほど作成した IAM ロールを探して選択します。  

![](./imgs/015_role_search.png)  

IAM ロールの詳細情報が表示されるので、ARN の隣にあるコピーボタンをクリックして ARN をコピーします。
![](./imgs/016_role_arn_copy.png)  

コピペした値を置き換えて role の値を設定します。

In [1]:
role = "arn:aws:iam::000000000000:role/StudioLabWhisperExecutionRole"  # TODO: コピペした値で置き換える

## Wav2Vec のデプロイ

まずは、 SageMaker SDK の HuggingFace 拡張を使って簡単にモデルをデプロイしてみましょう。  
以下のページで公開されている「wav2vec2-large-960h」と呼ばれているモデルを使っていきます。  

- https://huggingface.co/facebook/wav2vec2-large-960h

この HuggingFace のページ上で SageMaker でモデルをデプロイするためのコードを手軽に生成できます。  

まずは、ページにある「Deploy」ボタンをクリックします。  

![](./imgs/101_wav2vec_deploy_button.png)

いくつかデプロイの選択肢が出てくるので今回は「Amazon SageMaker」を選択します。  

![](./imgs/102_wav2vec_deploy_select_sagemaker.png)

Task を「Automatic Speech Recognition」、Configuration を「AWS」に設定すると deploy 用のコードが生成されます。  

![](./imgs/103_wav2vec_deploy_generate_code.png)



コピーしたコードはデプロイのコードと推論のコードが含まれています。  
このままでは動かないため、audio_serializer の追加や IAM Role 部分のコメントアウトなどをする必要があります。  

ここで使用している `DataSerializer` は推論リクエストの際に、音声ファイルをリクエストする手順を簡易化してくれます。  


注意！！！  
**以下のコードを実行すると、SageMaker 上でエンドポイントがデプロイされます。  
エンドポイントは時間課金されるため、注意してください。**  
意図せぬ課金を避けるためにも、最後の後片付けのステップは必ず実施してください。  

In [6]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.serializers import DataSerializer
import sagemaker

#### <変更箇所> 下の行はコメントアウトし、最初に設定した値を使う。 ####
# role = sagemaker.get_execution_role()
#### 
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'facebook/wav2vec2-base-960h',
	'HF_TASK':'automatic-speech-recognition'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.17.0',
	pytorch_version='1.10.2',
	py_version='py38',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
#### <変更箇所> 音声データをリクエストできるよう Audio Seriealizer を追加する ####
audio_serializer = DataSerializer(content_type='audio/x-audio')  # x-audio にしておくことで複数の音声フォーマットに対応
####
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge', # ec2 instance type
    #### <変更箇所> Audio Serializer を追加 ####
    serializer=audio_serializer
    ####
)

#### <変更箇所> 推論部分は別途実装する必要があるのでコメントアウト
# predictor.predict({
# 	'inputs': "sample1.flac"
# })
####



-----!

### デプロイしたエンドポイントに対して音声データを送ってみる

まずは、公開されている音声データを使ってどんな結果が返ってくるかみてみましょう。

huggingface.io 上で公開されている `libirispeech` と呼ばれる音声データセットの中からファイルを取得します。

In [7]:
!wget https://cdn-media.huggingface.co/speech_samples/sample1.flac

--2022-11-04 05:13:13--  https://cdn-media.huggingface.co/speech_samples/sample1.flac
Resolving cdn-media.huggingface.co (cdn-media.huggingface.co)... 108.156.184.85, 108.156.184.79, 108.156.184.2, ...
Connecting to cdn-media.huggingface.co (cdn-media.huggingface.co)|108.156.184.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 282378 (276K) [audio/flac]
Saving to: 'sample1.flac.1'

sample1.flac.1      100%[===================>] 275.76K  --.-KB/s    in 0.003s  

2022-11-04 05:13:13 (95.1 MB/s) - 'sample1.flac.1' saved [282378/282378]



In [10]:
audio_path = "sample1.flac"

res = predictor.predict(data=audio_path)
print(res)

{'text': "GOING ALONG SLUSHY COUNTRY ROADS AND SPEAKING TO DAMP AUDIENCES IN DRAUGHTY SCHOOL ROOMS DAY AFTER DAY FOR A FORTNIGHT HE'LL HAVE TO PUT IN AN APPEARANCE AT SOME PLACE OF WORSHIP ON SUNDAY MORNING AND HE CAN COME TO US IMMEDIATELY AFTERWARDS"}


また、音声データをバイナリ形式で送信する方法もあります。

In [11]:
audio_path = "sample1.flac"

with open(audio_path, "rb") as data_file:
    audio_data = data_file.read()
    res = predictor.predict(data=audio_data)
    print(res)

{'text': "GOING ALONG SLUSHY COUNTRY ROADS AND SPEAKING TO DAMP AUDIENCES IN DRAUGHTY SCHOOL ROOMS DAY AFTER DAY FOR A FORTNIGHT HE'LL HAVE TO PUT IN AN APPEARANCE AT SOME PLACE OF WORSHIP ON SUNDAY MORNING AND HE CAN COME TO US IMMEDIATELY AFTERWARDS"}


### 後片付け

先ほど作成したモデルとデプロイしたエンドポイントを最後に削除します。

In [12]:
predictor.delete_model()
predictor.delete_endpoint()